In [1]:
~1

-2

In [39]:
10**18

1000000000000000000

In [37]:
for i in range(20):
    print(i, bit_not(i))

0 1
1 0
2 1
3 0
4 3
5 2
6 1
7 0
8 7
9 6
10 5
11 4
12 3
13 2
14 1
15 0
16 15
17 14
18 13
19 12


In [38]:
for i in range(20):
    print(i, i*2)

0 0
1 2
2 4
3 6
4 8
5 10
6 12
7 14
8 16
9 18
10 20
11 22
12 24
13 26
14 28
15 30
16 32
17 34
18 36
19 38
